<a href="https://colab.research.google.com/github/coldsober-irene/hackathon1/blob/main/Copy_of_hackathon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  -U easynmt

In [ ]:
!pip install uvicorn fastapi

In [ ]:
!pip install fastapi pyngrok uvicorn nest-asyncio

In [40]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
# config the account
!ngrok authtoken 2HJPvK5e4VxcduBjO5XSc0LqqCR_5zV3b5gBcKuNMUnxdpfuP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [45]:
# REQUIRED IMPORTS
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json
import numpy as np
from easynmt import EasyNMT
import json # to access languages codes
from fastapi import FastAPI, Response
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
# change the default limit of pandas columns to display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% **WEBSCRAPING SECTION** %%%%%%%%%%%%%%%%%%%%%

In [28]:
pd.set_option('display.max_columns', None)

In [29]:
# DATA MINING
class dataMining:
  def __init__(self):
    pass
    
  # SOUP
  def Soup(self, url):
    return bs(requests.get(url).content, "html.parser")
  
  def get_text(self):
    soup = self.Soup(url = 'https://www.jobinrwanda.com/jobs/featured')
    parent = "https://www.jobinrwanda.com"
    co_links = {}
    app_link = []
    full_text = [[(p.get_text(), self.Soup(j_link).find("a", class_ = "btn btn-sm btn-success")) for p in dv.find_all('p', class_ = 'text-align-justify')] for j_link in [job_link for job_link in [parent+link['href'] if '/job/' in link['href'] else co_links.setdefault(link.text, parent+link['href']) 
    for div in soup.find_all('div', class_ = 'card-body p-2')[4] for link in div.find_all('a')] if '/job/' in job_link]
    for dv in self.Soup(j_link).find_all('div', class_ = "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
    ]
    return (full_text, app_link)

  def get_links(self):
    # GET SOUP OBJECT
    # soup = self.Soup(url = "https://www.jobinrwanda.com/jobs/all")
    soup = self.Soup(url = 'https://www.jobinrwanda.com/jobs/featured')
    # GET ALL JOBS LINKS
    company_infoLink = []
    
    all_jobsLinks = ["https://www.jobinrwanda.com"+link['href'] if "job" in link['href'] 
                     else company_infoLink.append(link.text) 
                     for div in soup.find_all("div", class_= "card-body p-2") 
                     for link in div.find_all("a")
                     ]
    all_jobsLinks = [link for link in all_jobsLinks if link]
    return (all_jobsLinks, company_infoLink)

  def get_jobInfo(self):
    jobLinks = self.get_links()[0]
    info = {}
    # REFINE THE INFO INTO A DICTIONARY
    def refine_info(ls):
      refined_info = {}
      infoZeroPattern = re.compile("\d+.*")
      views = infoZeroPattern.search(ls[0]).group()
      refined_info["views"] = views
      # OTHER DETAILS EXCEPT THE ONE ON POSITION 0
      otherInfoPattern = re.compile(":")
      for detail in ls[1:]:
        if detail.lower() == "apply":
          pass
        else:
          try:
            detailed = otherInfoPattern.split(detail)
            refined_info[detailed[0]] = detailed[1]
          except IndexError:
            pass
      return refined_info
       
    for index,url in enumerate(jobLinks):
      soup = self.Soup(url)
      pattern = re.compile("\s{2,}")
      job_info = [pattern.sub(" ",li.text.strip().replace("\n", "")) for ul in soup.find_all('ul', class_ = "list-group list-group-flush") 
      for li in ul.find_all('li')]
      # DICTIONANRY OF INFO
      info[index] = refine_info(job_info[:9])
    return info

  def get_description(self):
    jobLinks = self.get_links()[0]
    all_text= {}
    for index,link in enumerate(jobLinks):
      soup = self.Soup(link)
      # JOB TITLE
      job_title = soup.find('span', class_ = "field field--name-title field--type-string field--label-hidden")
      
      tags_content = soup.find_all('div', class_= "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
      
      for div in tags_content:
        temp = []
        try:
          # GET APPLICATION LINK
          app_link = soup.find("a", class_ = "btn btn-sm btn-success")['href']
          if app_link.startswith("/"):
            app_link = "https://www.jobinrwanda.com"+app_link
          temp.append(app_link)
          
        except TypeError:
          temp.append(np.nan)
        for tag in div.children:
          try:
            temp.append(tag.get_text())
          except (AttributeError, TypeError):
            pass
        temp.insert(0, job_title.text)
        all_text[index] = temp
    return all_text

In [30]:
# MERGE JOB DESCRIPTION WITH THE JOB INFO
def final_textDemo():
  global info
  # CLASS OBJECT CREATION
  jobInRwanda = dataMining()
  job_text = jobInRwanda.get_description()
  info = jobInRwanda.get_jobInfo()

  final_demo = []
  for index in list(info.keys()):
    try:
      info_value = list(info[index].values())
      # ADD COMPANY HIRING
      co_info = jobInRwanda.get_links()[1]
      info_value.append(co_info[index])
      # INCLUDE JOB TITLE
      info_value.append(job_text[index][0])
      # INCLUDE APPLY LINK
      info_value.append(job_text[index][1])
      # JOIN PARAGRAPHS 
      t = '\n'.join(job_text[index][2:])
      g = re.compile("\n|\\xa0").sub("", t)
      info_value.append(g)
      final_demo.append(info_value)
    except KeyError:
      pass
  return final_demo
# FUNCTION CALL
final_content = final_textDemo()

In [31]:
# FORM DATAFRAME
titles = list(info[0].keys()) + ["Company Hiring",'Job Title', "Application link", "Job Description"]
len(final_content[0])
# , titles
df = pd.DataFrame(data = final_content, columns = titles)


# GET IT & COMPUTER RELATED JOBS, INTERNERSHIPS AND CONSULTANCY

In [32]:
# prepare kewords IT & COMPUTER jobs
IT_keywords = """
                #informationtechnology #technology #it #cybersecurity #tech #computerscience #programming #business #coding #innovation #software #python #information #computer #informationsecurity #security #technologynews #java #networking #hacking #programmer #linux #technologyrocks #coder #technologythesedays #cloudcomputing #education #engineering #itservices #newtechnology
              """.split('#')[1:-2]
# print(IT_keywords)
IT_keywords[2] = IT_keywords[2].upper()
keywords = re.sub('\s',"","|".join(IT_keywords))

In [33]:
keywords = {'Job Description':keywords, 'Sector':'Computer|IT'}
df2 = df.copy()
df2['Is-IT-Job'] = False
df2['keywords found'] = None
def isIT_Job(targetCol = "Sector"):
  pattern = re.compile(f"{keywords[targetCol]}")
  for index in range(len(df2)):
    try:
      if len(pattern.findall(df2.iloc[index][targetCol])) >= 5 and targetCol == "Job Description":
        df2['Is-IT-Job'][index] = True
        df2['keywords found'][index] = set(pattern.findall(df2.iloc[index][targetCol]))
      elif pattern.findall(df2.iloc[index][targetCol]) and targetCol == "Sector":
        df2['Is-IT-Job'][index] = True
        df2['keywords found'][index] = set(pattern.findall(df2.iloc[index][targetCol]))
    except TypeError:
      pass
isIT_Job()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
# df2[df2['Is-IT-Job'] == True]

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% **TRANSLATOR SECTION** %%%%%%%%%%%%%%%%%%%%%

In [55]:
from starlette.responses import HTMLResponse
# try api 
app = FastAPI(title = 'display dataframe')
@app.get("/IT")
def show_it_jobs(orient:str='split'):
  return Response(df2.to_json(orient = orient), media_type = "application/json")

@app.get("/all_info")
def show_it_jobs(orient:str='split'):
  return Response(df.to_json(orient = orient), media_type = "application/json")

@app.get("/umucyo")
def show_it_jobs(orient:str='split'):
  return Response(df.to_json(orient = orient), media_type = "application/json")

model = EasyNMT("opus-mt")
@app.get("/translate", response_class = HTMLResponse)
def support_langs(lang:str=''):
  if not lang:
    with open('/content/drive/MyDrive/NLP fellowship/files/langs.json', 'r') as f:
      content = f.read()
      json_str = json.loads(content)
    return Response(content, media_type = "application/json")
  else:
    previous_lang = 'en'
    for row in range(len(df2)):
      try:
        df2['Job Description'][row] = model.translate([df2.iloc[row]['Job Description']], source_lang = previous_lang, target_lang=lang)
        df2['Job Title'][row] = model.translate([df2.iloc[row]['Job Title']], source_lang = 'en', target_lang=lang)
        previous_lang = lang
      except AttributeError:
        message = f"""<html>
                      <h1>{lang} is not among the languages supported by easyNMT.<br>Try another language</h1>
                      </html>
                    """
        return HTMLResponse(message)
      return Response(df2.to_json(orient = 'index'), media_type = 'application/json')
    
      

In [ ]:
df2.iloc[4]['Job Description']

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print("[REST API STARTED]")
print("[YOUR PUCLIC API URL]: ", ngrok_tunnel.public_url)
# 
nest_asyncio.apply()
uvicorn.run(app, port = 8000)

In [37]:
class translator:
  def __init__(self):
    # INITIALIZE THE MODEL FOR TRANSLATE
    self.model = EasyNMT("opus-mt")
  
  def supportedLangs(self, all_langs, supported):
    with open(all_langs) as f_json:
      content = f_json.read()
    langs = json.loads(content)
    print("langs: ", langs)

    with open(supported) as f2_json:
          content = f2_json.read()
    supplangs = json.loads(content)
    supp_codes = set(list(langs.values())).intersection(set(supplangs))
    supp_keyValue = {}
    for key in langs.keys():
      if langs[key] in supp_codes:
        supp_keyValue[key] = langs[key]
    return supp_keyValue
    

  def translate(self, sentences, targetLang, source = 'en', single = True):
    fail_message = "[FAIL]: destination lang is not a suported language by the model"
    success = 0
    if single:
      try:
        translation = self.model.translate(sentences, source_lang = source, target_lang = targetLang)
        return translation
      except:
        print(fail_message)
    else:
      translated = []
      for target in targetLang:
        try:
          translation = self.model.translate(sentences, source_lang = source, target_lang = target)
          translated.append(translation)
          success += 1
        except:
          print(fail_message)
      return translated, success


nest_asyncio : helps to make fastapi asynchronous

In [38]:
def umucyo_data():
  c = "https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do"
  s = bs(requests.get(c).content, 'html.parser')
  table = s.find('table', class_ = "article_table mb10")
  heads = [re.compile('[\r|\n|\t]+').sub('#', head.text.strip()) for head in table.find_all('thead')]
  heads = heads[0].split('#')
  rows = table.find_all('tr')
  full = [[detail.text.strip() for detail in row.find_all('td')][1:] for row in rows]

  # df
  return pd.DataFrame(full, columns = heads)